<a href="https://colab.research.google.com/github/sjnaj/DeepLearning/blob/master/Convolution/basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

卷积神经网络


多层感知机十分适合处理表格数据，其中行对应样本，列对应特征。 对于表格数据，我们寻找的模式可能涉及特征之间的交互，但是我们不能预先假设任何与特征交互相关的先验结构。 此时，多层感知机可能是最好的选择，然而对于高维感知数据，这种缺少结构的网络可能会变得不实用，会产生大量的参数，难以计算




适合于计算机视觉的神经网络架构：

平移不变性（translation invariance）：不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的反应，即为“平移不变性”。

局部性（locality）：神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远区域的关系，这就是“局部性”原则。最终，可以聚合这些局部特征，以在整个图像级别进行预测。

使用$[\mathbf{X}]_{i, j}$
和$[\mathbf{H}]_{i, j}$
分别表示输入图像和隐藏层中位置（i,j）处的像素

将参数从权重矩阵（如同我们先前在多层感知机中所做的那样）替换为四阶权重张量W

$$\begin{split}\begin{aligned} \left[\mathbf{H}\right]_{i, j} &= [\mathbf{U}]_{i, j} + \sum_k \sum_l[\mathsf{W}]_{i, j, k, l}  [\mathbf{X}]_{k, l}\\ &=  [\mathbf{U}]_{i, j} +
\sum_a \sum_b [\mathsf{V}]_{i, j, a, b}  [\mathbf{X}]_{i+a, j+b}.\end{aligned}\end{split}$$

$[\mathsf{V}]_{i, j, a, b} = [\mathsf{W}]_{i, j, i+a, j+b}$

V是由W重新索引下标所得

在a,b确定的矩形范围内求和

平移不变性:这意味着检测对象在输入X中的平移，应该仅导致隐藏表示H中的平移。也就是说，V和U实际上不依赖于(i,j)的值，即$[\mathsf{V}]_{i, j, a, b} = [\mathbf{V}]_{a, b}$

V被称为卷积核（convolution kernel）或者滤波器（filter）或感受野，亦或简单地称之为该卷积层的权重，通常该权重是可学习的参数。

滤波器的性质表明其有卷积和的筛选性质，从而得到所需的特征

局部性：在$|a|> \Delta$或$|b|> \Delta$的范围之外，我们可以设置$[\mathbf{V}]_{a, b} = 0$
。因此，我们可以将$[\mathbf{H}]_{i, j}$
重写为

$[\mathbf{H}]_{i, j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}.$

二维卷积和定义

$(f * g)(i, j) = \sum_a\sum_b f(a, b) g(i-a, j-b).$

这里不是使用$(i+a, j+b)$，而是使用差值。两者是旋转180度的区别。实际CNN中使用后者是为了便于计算，只需对应位置相乘，无须“卷着”乘，标准术语应为互相关(cross-correlation)而不是卷积

In [9]:
import torch
from torch import nn



In [10]:
def corr2d(X,K):
  '''计算二维互相关运算'''
  h,w=K.shape#卷积核高宽
  Y=torch.zeros(X.shape[0]-h+1,X.shape[1]-w+1)#结果矩阵的维度
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
  return Y

In [11]:
X=torch.arange(0.0,9).reshape(3,3)
K=torch.arange(0.0,4).reshape(2,2)
corr2d(X,K)


tensor([[19., 25.],
        [37., 43.]])

简单边缘检测

In [12]:
X=torch.ones(6,8)
X[:,2:6]=0
K=torch.tensor([[1.0,-1.0]])#检测水平方向的边缘(相邻一样为0)

In [13]:
Y=corr2d(X,K)#1是0到1，-1是1到0
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [14]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)#前两个参数是输入输出通道
X=X.reshape(1,1,6,8)

Y=Y.reshape(1,1,6,7)

for i in range(20):
  Y_hat=conv2d(X)
  l=(Y_hat-Y)**2
  conv2d.zero_grad()
  l.sum().backward()
  conv2d.weight.data[:]-=3e-2*conv2d.weight.grad#学习率3e-2
  if(i+2)%2==0:
    print(f'batch{i+1}:,loss:{l.sum()}')
print(conv2d.weight)

batch1:,loss:5.690177917480469
batch3:,loss:1.4246468544006348
batch5:,loss:0.43152564764022827
batch7:,loss:0.15124990046024323
batch9:,loss:0.057673338800668716
batch11:,loss:0.022905252873897552
batch13:,loss:0.009261544793844223
batch15:,loss:0.003773350967094302
batch17:,loss:0.0015421808930113912
batch19:,loss:0.0006311155739240348
Parameter containing:
tensor([[[[ 0.9979, -1.0020]]]], requires_grad=True)


填充和步幅

假设输入形状为$n_h\times n_w$，卷积核形状为$k_h\times k_w$，那么输出形状将是$(n_h-k_h+1) \times (n_w-k_w+1)$。 因此，卷积的输出形状取决于输入形状和卷积核的形状。

在应用了连续的卷积之后，我们最终得到的输出远小于输入大小。这是由于卷积核的宽度和高度通常大于1所导致的

如果我们添加$p_h$行填充（大约一半在顶部，一半在底部）和$p_w$列填充（左侧大约一半，右侧一半），则输出形状将为$$(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1)$$

假设$k_h$是奇数，我们将在高度的两侧填充$p_h/2$行。宽度同理。 卷积神经网络中卷积核的高度和宽度通常为奇数，例如1、3、5或7。 选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。



In [15]:
import torch
from torch import nn


# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape



torch.Size([8, 8])

In [16]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

当垂直步幅为$s_h$、水平步幅为$s_w$时，输出形状为

$$\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.$$

如果设置了$p_h=k_h-1$和$p_w=k_w-1$,则输出形状将简化为$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$

更进一步，如果输入的高度和宽度可以被垂直和水平步幅整除，则输出形状将为
$(n_h/s_h) \times (n_w/s_w)$



In [17]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape


torch.Size([4, 4])

In [18]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

多通道

In [28]:
import torch

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起（按元素叠加）
    return sum(corr2d(x, k) for x, k in zip(X, K))
X=torch.stack((torch.arange(0.0,9).reshape(3,3),torch.arange(1.0,10).reshape(3,3)))#把二维数组在第0维上堆叠为三维（矩阵放在tuple或list里）
K=torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in(X,K)


torch.Size([2, 2, 2])


为了获得多个通道的输出，我们可以为每个输出通道创建一个形状为的卷积核张量$c_i\times k_h\times k_w$，这样卷积核的形状是$c_o\times c_i\times k_h\times k_w$

In [30]:
K = torch.stack((K, K + 1, K + 2), 0)


def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
    
corr2d_multi_in_out(X, K)#第一个通道的结果与先前输入张量X和多输入单输出通道的结果一致。


tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

计算复杂度：$O(c_ic_oK_hK_wm_hm_w)$


**在np**.Conv2d里前两个参数可直接设置通道数